In [1]:
# install dependencies
!pip install pandas pyexcel pyexcel-ods pyexcel-xlsx openpyxl

# Import required libraries
import pandas as pd
from pyexcel_ods import get_data
import os

# Helper function to load files into a DataFrame
def load_file(file_path):
    ext = os.path.splitext(file_path)[-1].lower()
    if ext == ".tsv":
        return pd.read_csv(file_path, sep='\t')
    elif ext == ".xlsx":
        return pd.read_excel(file_path)
    elif ext == ".ods":
        data = get_data(file_path)
        sheet = list(data.keys())[0]  # Use the first sheet
        df = pd.DataFrame(data[sheet][1:], columns=data[sheet][0])
        return df
    else:
        raise ValueError(f"Unsupported file type: {ext}")

  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached pyexcel-0.7.1-py2.py3-none-any.whl.metadata (63 kB)
  Using cached pyexcel_ods-0.6.0-py2.py3-none-any.whl.metadata (19 kB)
  Using cached pyexcel_xlsx-0.6.0-py2.py3-none-any.whl.metadata (16 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached numpy-2.2.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached lml-0.1.0-py2.py3-none-any.whl.metadata (7.6 kB)
  Using cached pyexcel_io-0.6.7-py2.py3-none-any.whl.metadata (26 kB)
  Using cached texttable-1.7.0-py2.py3-none-any.whl.metadata (9.8 kB)
  Using cached odfpy-1.4.1-py2.py3-none-any.whl
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.ma

In [2]:
# Load supplier questions
questions_file = './supplier_questions.tsv' # './supplier_questions.ods' './supplier_questions.xlsx'
questions_df = load_file(questions_file)
print(f"Questions:\n{questions_df.head()}")

# Load context resources
context_file = './context.tsv'
context_df = load_file(context_file)
print(f"Context:\n{context_df.head()}")

# Concatenate questions and context resources
concatenated_questions = "\n".join(str(question) for question in questions_df['Question'])
concatenated_resources = "\n".join(str(res) for res in context_df['Resources'])

Questions:
                               Question
0  Is the supplier ISO 27001 certified?
1       Is the supplier SOC2 certified?
Context:
                                           Resources
0  https://sentry.io/legal/dpa/5.1.0/in-app/?user...
1                        https://sentry.io/security/
2  https://sentry.io/legal/terms/3.0.0/in-app/?us...


In [3]:
# create a base prompt to extend
base_prompt = """
Below is a list of questions and resources.
Please answer the questions based on the resources provided:
"""

In [4]:
# complete the base prompt with questions and related resources
prompt = f"{base_prompt}\n\nQuestions:\n{concatenated_questions}\n\nResources:\n{concatenated_resources}"
print(f"prompt:\n{prompt}")

prompt:

Below is a list of questions and resources.
Please answer the questions based on the resources provided:


Questions:
Is the supplier ISO 27001 certified?
Is the supplier SOC2 certified?

Resources:
https://sentry.io/legal/dpa/5.1.0/in-app/?userCurrentVersion=5.1.0
https://sentry.io/security/
https://sentry.io/legal/terms/3.0.0/in-app/?userCurrentVersion=3.0.1


In [5]:
# define string "constants" used in the anthropic system guidelines
add_notes = "additional notes"
url_resources = "URL resources"

# anthropic system guidelines 
anthropic_system = f"""
You're an ISO leading a compliance team seeking information about a potential future supplier. 
Your team is asked to perform a compliance due diligence. 
Respond in short and clear explanations. 
Please include {add_notes} and {url_resources} if available.
In case you can share {add_notes}, please add '{add_notes}:\n' as headline for the {add_notes} section.
In case you can share {url_resources}, please add '{url_resources}:\n' as headline for the {url_resources} section.
Don't be lazy!
"""

In [6]:
# install dependencies
!pip install anthropic
!pip install python-dotenv
!pip install pyexcel-ods3

  Using cached anthropic-0.42.0-py3-none-any.whl.metadata (23 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.8.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.2 kB)
  Using cached pydantic-2.10.4-py3-none-any.whl.metadata (29 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.27.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
Using cached anthropic-0.42.0-py3-none-any.whl (203 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached jiter-0.8.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (345 kB)
Using cached pydantic-2.10.4-py3-none-any.whl (431 kB)
Using cached pydantic_core-2.27.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1

In [7]:
# read the anthropic api key from .env file
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('ANTHROPIC_API_KEY')
if api_key is None:
    raise ValueError("The 'ANTHROPIC_API_KEY' environment variable is not set.")

In [8]:
# define a function to call claude using anthropic library
import anthropic
client = anthropic.Anthropic()

def ask(prompt):
    response = client.messages.create(
        model="claude-3-5-sonnet-latest",
        max_tokens=2000,
        temperature=0.0,
        system=anthropic_system,
        messages=[{"role": "user", "content": prompt}]
    )
    # Extract and return the text content
    if response.content:
        text_response = response.content[0].text  # Assuming `response.content` is a list with a `TextBlock`
        return text_response
    else:
        print("Empty response.content")
        return ""

In [9]:
# request and fetch claude's answers and match questions and answers
import pyexcel

res = None
try:
    res = ask(prompt)
    if not res.strip():  # Handle empty responses
        print("Empty response, exiting.")

except Exception as e:
    print(f"Exception occurred: {e}")
    print(f"Exception type: {type(e)}")

if res is not None:
    
    print(f"---------------/Claude's answers ----------------:\n {str(res)}\n---------------/end of Claude's answers ----------------\n\n\n")
    
    # Define the file path for saving the raw result
    raw_result_file = "claude_answers.txt"
    
    # Save the raw result to the text file
    with open(raw_result_file, 'w', encoding='utf-8') as file:
        file.write(res)  # Write the raw result to the file
    
    print(f"Claude's answers saved to: {raw_result_file}")
    
    response_lines = res.split("\n")
    answers = {}
    answer_cntr = 0
    
    # Parse responses
    for line in response_lines:
        if ":" in line:
            key, value = line.split(":", 1)
            if key == add_notes or key == url_resources:
                pass
            elif value.strip() != "" and answer_cntr < len(questions_df['Question']):
                answers[key.strip()] = value.strip()
                answer_cntr += 1
    
    # Build output DataFrame
    output_data = {
        "Question": concatenated_questions.split("\n"),
        "Answer": list(answers.values())
    }
    output_df = pd.DataFrame(output_data)
    
    output_tsv = "QandA.tsv"
    output_excel = "QandA.xlsx"
    output_ods = "QandA.ods"
    
    # Save to TSV, Excel, and ODT
    output_df.to_csv(output_tsv, sep='\t', index=False)
    output_df.to_excel(output_excel, index=False)
    output_df.to_csv(output_ods, sep=',', index=False)  # use ODS-specific library if needed

    # Convert DataFrame to a list of lists for pyexcel compatibility
    data_to_save = [output_df.columns.tolist()] + output_df.values.tolist()
    # Save to an ODS file
    pyexcel.save_as(array=data_to_save, dest_file_name=output_ods)

    print(f"Questions and answers saved to: {output_tsv}, {output_excel}, {output_ods}")

---------------/Claude's answers ----------------:
 Based on the provided resources:

ISO 27001: Yes, Sentry is ISO 27001 certified.

SOC2: Yes, Sentry maintains SOC 2 Type II certification.

Additional notes:
- Sentry undergoes regular third-party audits to maintain these certifications
- Their security certifications are verified annually
- They maintain a comprehensive security program that includes both technical and organizational measures
- Certifications cover their cloud-based error monitoring platform and related services

URL resources:
- Security overview: https://sentry.io/security/
- You can request their security certifications and audit reports through their sales team or customer support
- Their security documentation and certifications can be accessed through their Trust Center after signing an NDA
---------------/end of Claude's answers ----------------



Claude's answers saved to: claude_answers.txt
Questions and answers saved to: QandA.tsv, QandA.xlsx, QandA.ods
